In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
!pip install -r requirements.txt
from src.preprocessing import DataPreprocessor
from src.model import Model
from src.gui import *

ERROR: Could not find a version that satisfies the requirement sklearn==1.6.1 (from versions: 0.0, 0.0.post1, 0.0.post2, 0.0.post4, 0.0.post5, 0.0.post7, 0.0.post9, 0.0.post10, 0.0.post11, 0.0.post12)
ERROR: No matching distribution found for sklearn==1.6.1
You should consider upgrading via the 'C:\Users\csbro\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Load training and testing datasets
dataset = DataPreprocessor(
    filepath="data/Line Listing(dxjZy).xlsx",
    column_names=['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']
)
dataset.explode_column("Suspect Product Active Ingredients", ";")

print("Value Counts:")
print(dataset.get_value_counts("Serious"))

print("\nData Types:")
print(dataset.get_dataframe().dtypes)

dataset.get_dummies(["Suspect Product Active Ingredients"])
dataset.data.groupby("Case ID").max().reset_index()
dataset.drop_columns(["Case ID"])

dataset.ensure_numeric_column("Patient Age", decimal=False)

dataset.convert_nulls("Patient Weight", nulls=["Not Specified"], output="0 KG")
dataset.ensure_numeric_column("Patient Weight")

dataset.encode_column("Sex")
dataset.encode_column("Serious")

dataset.drop_all_nulls()

print("\nNew Value Counts:")
print(dataset.get_value_counts("Serious"))

print("\nDataset Head:")
print(dataset.get_dataframe().head())

x_train, x_test, y_train, y_test = dataset.get_standardised_train_test_split(
    dataset.data.columns, 
    y="Serious", 
    test_size=0.2, 
    random_state=42
)

Value Counts:
Serious
Serious        3199
Non-Serious      67
Name: count, dtype: int64

Data Types:
Case ID                                int64
Suspect Product Active Ingredients    object
Serious                               object
Sex                                   object
Patient Age                           object
Patient Weight                        object
dtype: object

New Value Counts:
Serious
1    3199
0      67
Name: count, dtype: int64
    Case ID  Serious  Sex  Patient Age  Patient Weight  .Alpha.-Tocopherol  \
0  24750965        1    1           75               0               False   
0  24750965        1    1           75               0               False   
1  24391183        1    0           73               0               False   
1  24391183        1    0           73               0               False   
1  24391183        1    0           73               0               False   

   .Alpha.-Tocopherol Acetate, Dl-  .Alpha.-Tocopherol, Dl-  \
0         

In [16]:
# Running model
input_dim = len(x_train[0])
model = Model(input_dim)
model.train_model(x_train, y_train)
model.evaluate_model(x_test, y_test)

c:\Users\csbro\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\csbro\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 0.3529814733816992
Epoch 2/50, Loss: 0.04888769319968116
Epoch 3/50, Loss: 0.023824358433513423
Epoch 4/50, Loss: 0.02150281560543368
Epoch 5/50, Loss: 0.020844572054423822
Epoch 6/50, Loss: 0.021197562659103934
Epoch 7/50, Loss: 0.021021584908494997
Epoch 8/50, Loss: 0.02139682893282109
Epoch 9/50, Loss: 0.02109333717923685
Epoch 10/50, Loss: 0.021767085413491104
Epoch 11/50, Loss: 0.021516268851772154
Epoch 12/50, Loss: 0.022445985628575905
Epoch 13/50, Loss: 0.0216074303043126
Epoch 14/50, Loss: 0.02129404672752305
Epoch 15/50, Loss: 0.021229135334667772
Epoch 16/50, Loss: 0.021350405773108188
Epoch 17/50, Loss: 0.022306336138238448
Epoch 18/50, Loss: 0.021853199928201075
Epoch 19/50, Loss: 0.021520888029665845
Epoch 20/50, Loss: 0.021763773560140667
Epoch 21/50, Loss: 0.02127471598001515
Epoch 22/50, Loss: 0.02179746439172205
Epoch 23/50, Loss: 0.021557411882762296
Epoch 24/50, Loss: 0.021497997305520652
Epoch 25/50, Loss: 0.02191920065081099
Epoch 26/50, Loss: 0.

c:\Users\csbro\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([654])) that is different to the input size (torch.Size([654, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
